In [89]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from sklearn import cross_validation as cv

pd.options.display.max_rows = 10
pd.options.display.max_columns = 15

data = pd.read_table('data/u.data', sep='\t', usecols=['user id', 'movie id', 'rating'])
data

,user id,movie id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1
...,...,...,...
99995,880,476,3
99996,716,204,5
99997,276,1090,1
99998,13,225,2


In [2]:
import jupyter-spark

SyntaxError: invalid syntax (<ipython-input-2-899b705823f4>, line 1)

In [90]:
train_data, test_data = cv.train_test_split(data, train_size=0.8)

A = train_data.pivot_table(index='user id', columns='movie id', values='rating', fill_value=0)
B = test_data.pivot_table(index='user id', columns='movie id', values='rating', fill_value=0)
A

movie id,1,2,3,4,5,6,7,...,1675,1676,1677,1679,1680,1681,1682
user id,,,,,,,,,,,,,,,
1,5,3,0,3,3,5,4,...,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0
5,4,3,0,0,0,0,0,...,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0
940,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0
941,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0


In [91]:
# Index matrix for training data
R1 = A.copy()
R1[R1 > 0] = 1

# Index matrix for test data
R2 = B.copy()
R2[R2 > 0] = 1

R1

movie id,1,2,3,4,5,6,7,...,1675,1676,1677,1679,1680,1681,1682
user id,,,,,,,,,,,,,,,
1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0
5,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0
940,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0
941,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0


In [92]:
lmbda = 0.1 # Regularisation weight
k = 20 # Dimensionality of latent feature space
m, n = A.shape # Number of users and items
n_epochs = 5 # Number of epochs

U = 3 * np.random.rand(k,m) # Latent user feature matrix
M = 3 * np.random.rand(k,n) # Latent movie feature matrix
M[0,:] = A[A != 0].mean(axis=0) # Avg. rating for each movie
E = np.eye(k) # (k x k)-dimensional idendity matrix
train_errors = []
test_errors = []
U

array([[ 1.6380718 ,  1.08536708,  2.55953983, ...,  0.18460293,
         1.08580345,  1.83008185],
       [ 0.94753421,  0.59250129,  2.19725727, ...,  2.48924501,
         2.06526922,  1.02253475],
       [ 0.01712434,  0.88436999,  1.28806171, ...,  1.18872572,
         0.81750143,  1.7991358 ],
       ..., 
       [ 0.14735808,  1.27789944,  0.51309043, ...,  0.04843655,
         0.93818939,  2.10019522],
       [ 2.75082124,  1.85988662,  1.49623803, ...,  1.02850016,
         1.05590672,  1.1534317 ],
       [ 0.4402032 ,  0.48285361,  1.02246843, ...,  0.26722795,
         0.89670208,  0.06779873]])

In [93]:
# Calculate the RMSE
def rmse(R1, A, M, U):
    return np.sqrt(np.sum((R1 * (A - np.dot(U.T, M)))**2)/len(A[A > 0]))

In [117]:
train_errors = []
test_errors = []
'''
# Repeat until convergence
for epoch in list(range(n_epochs)):
    # Fix Q and estimate P
    for i, Ri in R1.iterrows():
        nui = np.count_nonzero(Ri) # Number of items user i has rated
        if (nui == 0): nui = 1 # Be aware of zero counts!
    
        # Least squares solution
        Ai = np.dot(M, np.dot(np.diag(Ri), M.T)) + lmbda * nui * E
        Vi = np.dot(M, np.dot(np.diag(Ri), A[i].T))
        U[:,i] = np.linalg.solve(Ai,Vi)
        
    # Fix P and estimate Q
    for j, Rj in R1.T.iterrows():
        nmj = np.count_nonzero(Rj) # Number of users that rated item j
        if (nmj == 0): nmj = 1 # Be aware of zero counts!
        
        # Least squares solution
        Aj = np.dot(U, np.dot(np.diag(Rj), U.T)) + lmbda * nmj * E
        Vj = np.dot(U, np.dot(np.diag(Rj), A[:,j]))
        M[:,j] = np.linalg.solve(Aj,Vj)

    train_errors.append(rmse(R1, A, M, U))
    test_errors.append(rmse(R2, B, M, U))
    
    print("[Epoch {0}/{1}] train error: {2}, test error: {3}".format(epoch+1, n_epochs, train_rmse, test_rmse))
    
print("Algorithm converged")
'''
for i, Ri in R1.iterrows():
    print( A.shape )
A

(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943, 1654)
(943

movie id,1,2,3,4,5,6,7,...,1675,1676,1677,1679,1680,1681,1682
user id,,,,,,,,,,,,,,,
1,5,3,0,3,3,5,4,...,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0
5,4,3,0,0,0,0,0,...,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0
940,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0
941,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0
